In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [6]:
exa = Agilent_EXA_N9010A("EXA")
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
#ro_lo = EXG("EXG")

In [7]:
q_lo = MXG("MXG")

In [37]:
ro_lo = Agilent_PNA_L("PNA-L1")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [9]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [38]:
ro_resonator_frequency = 7.535e9
if_offset = 0

In [42]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "Xmons_Nb_bandage_res_mixer", -10)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=7,
                    ssb_power=-22, waveform_resolution=1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Calibration data for mixer Xmons_Nb_bandage_res_mixer
Mixer parameters: {'mixer_id': 'Xmons_Nb_bandage_res_mixer', 'iq_attenuation': -10}
Radiation parameters: {'lo_frequency': 7535000000.0, 'lo_power': 7, 'if_frequency': 0, 'ssb_power': -22, 'waveform_resolution': 1}
Optimization results: {'dc': -96.054351806640625, 'dc_open': [-22.02252197265625]}
Optimization parameters {'dc_offsets': array([ 0.0420697 , -0.09860207]), 'dc_offsets_open': array([ 0.29668933,  0.29614821]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 23.53 s
Finished at: 2017-08-08 20:21:46.452112


In [45]:
dm.save_IQMX_calibration(res)

In [15]:
calibrations=dm.load_IQMX_calibration_database("Xmons_Nb_bandage_res_mixer", -10)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

In [18]:
calibrations

{frozenset({('if_frequency', 50000000.0),
            ('lo_frequency', 7535000000.0),
            ('lo_power', 7),
            ('ssb_power', -22),
            ('waveform_resolution',
             1)}): <lib.iq_mixer_calibration.IQCalibrationData at 0x9b33898>}

## Qubit

In [22]:
qubit_frequency = 6.16e9
if_frequency = 50e6

In [25]:
cal = IQCalibrator(q_awg, exa, q_lo, "Xmons_Nb_bandage_qub_mixer", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=7,
                    ssb_power=-22, waveform_resolution=.1, iterations=5, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])}
print(res)

Calibration data for mixer Xmons_Nb_bandage_qub_mixer
Mixer parameters: {'mixer_id': 'Xmons_Nb_bandage_qub_mixer', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 6210000000.0, 'lo_power': 7, 'if_frequency': 50000000.0, 'ssb_power': -22, 'waveform_resolution': 0.1}
Optimization results: {'dc': -80.516433715820313, 'if': [-22.21625328063965, -59.436492919921875, -81.97730255126953]}
Optimization parameters {'dc_offsets': array([ 0.00710647, -0.03581821]), 'dc_offsets_open': None, 'if_offsets': array([ 0.00663281, -0.03863291]), 'if_amplitudes': array([ 0.24658357,  0.2248116 ]), 'if_phase': array([-1.60709686])}
Optimization time: 0 h 1 m 25.39 s
Finished at: 2017-08-08 18:46:28.556067


In [26]:
dm.save_IQMX_calibration(res)

In [ ]:
q_lo.visa_instr.close()

In [ ]:
q_awg.get_1st_delay()

In [ ]:
exa.set_singlesweep_mode()

In [ ]:
exa.set_centerfreq(6e9)

In [ ]:
data_u = exa.make_sweep_get_data()
freqs_u = exa.get_freqpoints()

In [ ]:
plt.plot(freqs_u,data_u)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(6.5, 4.5)
plt.annotate('$P_{LO}= 7$ dBm\n'+'$P_{SB}=-15$ dBm\n'+'Uncalibrated', xy=(0.7, 0.75), xycoords='axes fraction', fontsize=16)
plt.grid()
plt.savefig('./cal_pictures/uncal2.pdf', dpi=300)

In [ ]:
q_awg.output_continuous_wave(frequency=50e6, amplitude=1, phase=0, offset=0,
                                 waveform_resolution=1, channel=1)
q_awg.output_continuous_wave(frequency=50e6, amplitude=1, phase=0, offset=0,
                                 waveform_resolution=1, channel=2)

In [ ]:
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16

In [ ]:
q_lo.set_power(7)

In [ ]:
l = [1,2,3]
d = [4,5,6]

In [ ]:
l+[len(d)]

In [ ]:
for i,j in zip(l,d):
    print(i,j)

In [ ]:
import numpy as np

In [ ]:
np.product([[1,1,1],[1,1,1]])